In [ ]:
import sys
import os
import time
import threading
import queue
import re
import numpy as np
import pandas as pd
from scipy import stats
from scipy.io import loadmat
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.metrics import (confusion_matrix, accuracy_score, f1_score,
                             precision_score, recall_score, roc_auc_score,
                             matthews_corrcoef)
import joblib
import struct

# PyQt5 y Matplotlib para GUI y gráficos
from PyQt5 import QtWidgets, QtCore
from PyQt5.QtWidgets import QFileDialog, QMessageBox, QTableWidgetItem, QTabWidget, QInputDialog
from PyQt5.QtCore import pyqtSignal
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

# Intentaremos importar bleak (BLE). Si no está, usaremos modo simulación.
try:
    from bleak import BleakClient, BleakScanner
    BLEAK_AVAILABLE = True
except Exception:
    BLEAK_AVAILABLE = False

# Parámetros globales
BLE_MAC = "DD:70:A7:9C:C7:0F"
BLE_NAME = "WT901BLE67"
MODEL_PATH = "modelo_svm.joblib"

# ---------------------------------------------------------------------
# UTILIDADES DE PROCESAMIENTO (preprocesado, ventanas, features, selección)
# ---------------------------------------------------------------------

def normalize_colname(c):
    """Normaliza nombre de columna para comparar (quita espacios, paréntesis, minusculas)."""
    return re.sub(r'[^a-z0-9]', '', c.lower())

def encontrar_columna(cols, patrones):
    """
    Busca en cols una columna que contenga cualquiera de los patrones.
    `patrones` puede ser lista de strings como ['ax','accel','ax(g)'].
    """
    norm = [normalize_colname(c) for c in cols]
    for pat in patrones:
        p = normalize_colname(pat)
        for i, n in enumerate(norm):
            if p in n:
                return cols[i]
    # como último recurso, buscar por inicio de palabra (ej Acc, Gyr)
    for pat in patrones:
        p0 = pat[0].lower()
        for i, n in enumerate(norm):
            if n.startswith(p0):
                return cols[i]
    return None

def preprocesado(df):
    """
    Preprocesado: mapea columnas del CSV a [ax,ay,az,gx,gy,gz] usando nombres flexibles.
    Devuelve (arr, labels, df) donde arr es numpy (N, n_channels) y labels puede ser None.
    """
    df = df.dropna().reset_index(drop=True)
    cols = df.columns.tolist()
    print("Columnas detectadas:", cols)

    # patrones probables para cada canal (consideramos nombres como ax(g), wx(deg/s), etc.)
    ax_col = encontrar_columna(cols, ['ax', 'accel', 'accx', 'acc_x', 'ax(g)', 'axg'])
    ay_col = encontrar_columna(cols, ['ay', 'accy', 'acc_y', 'ay(g)'])
    az_col = encontrar_columna(cols, ['az', 'accz', 'acc_z', 'az(g)'])
    gx_col = encontrar_columna(cols, ['gx', 'gyro', 'wx', 'gyro_x', 'wx(deg/s)', 'wxdegs'])
    gy_col = encontrar_columna(cols, ['gy', 'gyro_y', 'wy', 'wy(deg/s)'])
    gz_col = encontrar_columna(cols, ['gz', 'gyro_z', 'wz', 'wz(deg/s)'])
    label_col = encontrar_columna(cols, ['label', 'etiqueta', 'target', 'class'])

    selected_cols = [c for c in [ax_col, ay_col, az_col, gx_col, gy_col, gz_col] if c is not None]
    if len(selected_cols) == 0:
        raise ValueError("No se detectaron columnas de acelerómetro/giroscopio. Revisa nombres de columnas.")
    # convertir a float (si es posible)
    try:
        arr = df[selected_cols].astype(float).values
    except Exception:
        # intentar limpiar caracteres comunes y convertir
        df_sel = df[selected_cols].applymap(lambda x: float(re.sub(r'[^\d\.\-eE]', '', str(x))) if pd.notnull(x) else np.nan)
        arr = df_sel.values

    labels = df[label_col].values if label_col is not None else None
    return arr, labels, df  # también devolvemos el DataFrame original para vista previa

def sliding_window(data, window_size, step):
    """Segmentación con ventanas superpuestas."""
    n = data.shape[0]
    windows = []
    for start in range(0, n - window_size + 1, step):
        win = data[start:start+window_size, :]
        windows.append(win)
    return windows

def extract_features_from_window(win):
    """Extrae features estadísticos por canal (mean,std,rms,skew,kurt,min,max)."""
    feats = []
    for ch in range(win.shape[1]):
        x = win[:, ch]
        mean = np.mean(x)
        std = np.std(x)
        rms = np.sqrt(np.mean(x**2))
        skew = float(stats.skew(x))
        kurt = float(stats.kurtosis(x))
        mn = np.min(x)
        mx = np.max(x)
        feats.extend([mean, std, rms, skew, kurt, mn, mx])
    return np.array(feats)

def extract_features(data, window_size=200, step=100):
    """Aplica sliding_window y extrae features para cada ventana."""
    wins = sliding_window(data, window_size, step)
    if len(wins) == 0:
        return np.zeros((0, 0))
    X = np.array([extract_features_from_window(w) for w in wins])
    return X

# --------------------------
# Entrenamiento / Guardado
# --------------------------

def crear_y_entrenar_modelo(X_train, y_train, k_best=20):
    """Crea pipeline: scaler -> SelectKBest -> SVC(probability=True) y guarda."""
    scaler = StandardScaler()
    selector = SelectKBest(score_func=f_classif, k=min(k_best, X_train.shape[1]))
    clf = SVC(kernel='rbf', probability=True, class_weight='balanced')
    pipe = Pipeline([('scaler', scaler), ('selector', selector), ('svc', clf)])
    pipe.fit(X_train, y_train)
    joblib.dump(pipe, MODEL_PATH)
    return pipe

def cargar_modelo():
    if os.path.exists(MODEL_PATH):
        return joblib.load(MODEL_PATH)
    return None

# --------------------------
# MÉTRICAS Y REPORTE
# --------------------------

def calcular_metricas(y_true, y_pred, y_proba=None):
    cm = confusion_matrix(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    mcc = matthews_corrcoef(y_true, y_pred) if len(np.unique(y_true))>1 else 0.0
    auc = None
    if y_proba is not None and y_proba.shape[1] > 1:
        try:
            auc = roc_auc_score(y_true, y_proba[:,1])
        except Exception:
            auc = None
    return {
        'confusion_matrix': cm,
        'accuracy': acc,
        'f1': f1,
        'precision': prec,
        'recall': rec,
        'mcc': mcc,
        'auc': auc
    }

# ---------------------------------------
# INTERFAZ GRÁFICA (PyQt5) - COMPONENTES
# ---------------------------------------

class MplCanvas(FigureCanvas):
    """Canvas matplotlib embebido"""
    def __init__(self, parent=None, width=5, height=3, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        super(MplCanvas, self).__init__(fig)

class MainWindow(QtWidgets.QMainWindow):
    # señal para informar estado BLE al hilo principal
    ble_status = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.setWindowTitle("Interfaz Pipeline Sensor - Proyecto Integrativo")
        self.setGeometry(100, 100, 1200, 760)

        # Cola para datos en vivo
        self.data_queue = queue.Queue()
        self.acquiring = False
        self.ble_client = None
        self.simulate = not BLEAK_AVAILABLE

        # Modelo
        self.model = cargar_modelo()
        if self.model is not None:
            print("Modelo cargado desde", MODEL_PATH)
        else:
            print("No se encontró modelo entrenado. Use 'Entrenar modelo' o cargue uno.")

        # Almacenes
        self.raw_data_buffer = []  # muestras crudas en lista
        self.loaded_df = None      # DataFrame original (cuando se carga archivo)
        self.plot_buffer = None    # buffer para plotting (N, n_channels)
        self.selected_device_address = None  # >>> CAMBIO: dirección seleccionada por usuario

        # Construir UI
        self._build_ui()

        # conectar señal BLE a handler (muestra mensajes)
        self.ble_status.connect(self._show_ble_message)

        # Timer para actualizar gráficos desde cola
        self.timer = QtCore.QTimer()
        self.timer.setInterval(200)  # ms
        self.timer.timeout.connect(self.actualizar_grafico_desde_cola)
        self.timer.start()

        self.show()

    def _build_ui(self):
        central = QtWidgets.QWidget()
        main_layout = QtWidgets.QVBoxLayout(central)

        # Botones de modos
        menu_layout = QtWidgets.QHBoxLayout()
        self.btn_modo_a = QtWidgets.QPushButton("Modo A: Cargar archivo")
        self.btn_modo_b = QtWidgets.QPushButton("Modo B: Adquisición en vivo")
        self.btn_modo_c = QtWidgets.QPushButton("Modo C: Tiempo real (ventanas)")
        menu_layout.addWidget(self.btn_modo_a)
        menu_layout.addWidget(self.btn_modo_b)
        menu_layout.addWidget(self.btn_modo_c)

        # Utilidades
        util_layout = QtWidgets.QHBoxLayout()
        self.btn_cargar = QtWidgets.QPushButton("Cargar .csv / .mat")
        self.btn_guardar_datos = QtWidgets.QPushButton("Guardar datos crudos")
        self.btn_entrenar = QtWidgets.QPushButton("Entrenar modelo ")
        # >>> CAMBIO: Botón para escanear BLE manualmente y seleccionar dispositivo
        self.btn_scan_ble = QtWidgets.QPushButton("Escanear BLE")
        util_layout.addWidget(self.btn_cargar)
        util_layout.addWidget(self.btn_guardar_datos)
        util_layout.addWidget(self.btn_entrenar)
        util_layout.addWidget(self.btn_scan_ble)

        # Área principal: gráfico + pestañas (datos crudos / features)
        area_layout = QtWidgets.QHBoxLayout()

        # Gráfico
        self.canvas = MplCanvas(self, width=7, height=5, dpi=100)
        area_layout.addWidget(self.canvas, 2)

        # Pestañas para mostrar datos
        right_panel = QtWidgets.QVBoxLayout()
        tabs = QTabWidget()
        # Tab 1: Datos crudos (vista previa)
        self.table_raw = QtWidgets.QTableWidget()
        tabs.addTab(self.table_raw, "Datos crudos (vista previa)")
        # Tab 2: Features y predicciones
        self.table_features = QtWidgets.QTableWidget()
        tabs.addTab(self.table_features, "Features / Predicciones")
        # Tab 3: Métricas
        self.metrics_text = QtWidgets.QTextEdit()
        self.metrics_text.setReadOnly(True)
        tabs.addTab(self.metrics_text, "Métricas")

        right_panel.addWidget(tabs)
        area_layout.addLayout(right_panel, 1)

        # Panel inferior: controles adquisición y parámetros
        bottom_layout = QtWidgets.QHBoxLayout()
        self.btn_start = QtWidgets.QPushButton("Iniciar adquisición")
        self.btn_stop = QtWidgets.QPushButton("Detener adquisición")
        self.btn_process_block = QtWidgets.QPushButton("Procesar bloque adquirido")
        bottom_layout.addWidget(self.btn_start)
        bottom_layout.addWidget(self.btn_stop)
        bottom_layout.addWidget(self.btn_process_block)

        # Parámetros
        params_layout = QtWidgets.QFormLayout()
        self.input_window = QtWidgets.QLineEdit("200")
        self.input_step = QtWidgets.QLineEdit("100")
        self.input_kbest = QtWidgets.QLineEdit("20")
        params_layout.addRow("Window (muestras):", self.input_window)
        params_layout.addRow("Step (muestras):", self.input_step)
        params_layout.addRow("SelectKBest k:", self.input_kbest)

        main_layout.addLayout(menu_layout)
        main_layout.addLayout(util_layout)
        main_layout.addLayout(area_layout)
        main_layout.addLayout(bottom_layout)
        main_layout.addLayout(params_layout)
        self.setCentralWidget(central)

        # Conexiones
        self.btn_cargar.clicked.connect(self.action_cargar_archivo)
        self.btn_guardar_datos.clicked.connect(self.action_guardar_datos)
        self.btn_entrenar.clicked.connect(self.action_entrenar_modelo)
        self.btn_start.clicked.connect(self.action_iniciar_adquisicion)
        self.btn_stop.clicked.connect(self.action_detener_adquisicion)
        self.btn_process_block.clicked.connect(self.action_procesar_bloque)
        self.btn_scan_ble.clicked.connect(self.action_scan_ble)  # >>> CAMBIO: conectar escaneo
        self.btn_modo_a.clicked.connect(lambda: QMessageBox.information(self, "Modo A", "Use 'Cargar .csv / .mat' para seleccionar un archivo."))
        self.btn_modo_b.clicked.connect(lambda: QMessageBox.information(self, "Modo B", f"Conectar BLE a {BLE_NAME} ({BLE_MAC})" + ("\n(Simulado porque bleak no está)" if self.simulate else "")))
        self.btn_modo_c.clicked.connect(lambda: QMessageBox.information(self, "Modo C", "Procesamiento en tiempo real con ventanas deslizantes y modelo entrenado."))

    # ---------------------------------------
    #Sensor
    # ---------------------------------------
    def _show_ble_message(self, msg):
        QMessageBox.information(self, "BLE", msg)

    # ---------------------------
    # Acciones: cargar / guardar
    # ---------------------------
    def action_cargar_archivo(self):
        options = QFileDialog.Options()
        fname, _ = QFileDialog.getOpenFileName(self, "Seleccionar archivo .csv o .mat", "", "CSV Files (*.csv);;MAT Files (*.mat);;All Files (*)", options=options)
        if not fname:
            return
        try:
            if fname.lower().endswith('.csv'):
                # Intentar detectar separador automáticamente
                try:
                    df = pd.read_csv(fname, sep=None, engine='python')
                except Exception:
                    df = pd.read_csv(fname)
            elif fname.lower().endswith('.mat'):
                mat = loadmat(fname)
                key = None
                for k in ['data', 'X', 'datos', 'signals']:
                    if k in mat:
                        key = k
                        break
                if key is None:
                    for k,v in mat.items():
                        if isinstance(v, np.ndarray):
                            key = k
                            break
                if key is None:
                    QMessageBox.warning(self, "Error", "No se encontró variable adecuada en .mat")
                    return
                arr = np.array(mat[key])
                # si arr es 2D con shape (N, M) convertir a df con nombres genéricos
                df = pd.DataFrame(arr)
            else:
                QMessageBox.warning(self, "Formato no soportado", "Solo .csv y .mat son soportados.")
                return

            # Preprocesado (mapear columnas)
            data, labels, df_original = preprocesado(df)
            if data is None or data.shape[0] == 0:
                QMessageBox.warning(self, "Error", "No se pudo extraer datos del archivo.")
                return

            # Guardar buffer y DataFrame
            self.raw_data_buffer = data.tolist()
            self.loaded_df = df_original  # guardamos DataFrame original

            # >>> CAMBIO: Llenar tabla raw y preparar plot_buffer para graficar LOS DATOS CARGADOS
            self._llenar_tabla_raw(df_original)
            # plot_buffer tendrá los datos cargados (numpy)
            self.plot_buffer = data.copy()

            # Extraer features
            window_size = int(self.input_window.text())
            step = int(self.input_step.text())
            X = extract_features(data, window_size=window_size, step=step)
            print("Ventanas extraídas:", X.shape)

            model = self.model
            y_pred = None
            y_proba = None
            if model is not None and X.shape[0] > 0 and X.shape[1] > 0:
                try:
                    y_proba = model.predict_proba(X)
                    y_pred = model.predict(X)
                except Exception as e:
                    print("Error prediciendo con el modelo:", e)
                    y_proba = None
                    y_pred = None

            # Mostrar tabla de features y predicciones
            if X.shape[0] > 0 and X.shape[1] > 0:
                self._llenar_tabla_features(X, y_pred, y_proba)
            else:
                self.table_features.clear()
                self.table_features.setRowCount(0)
                self.table_features.setColumnCount(0)

            # Si hay etiquetas en el archivo, calcular métricas
            if labels is not None and y_pred is not None:
                wins = sliding_window(labels.reshape(-1,1), window_size, step)
                y_true = np.array([stats.mode(w.flatten())[0][0] for w in wins])
                met = calcular_metricas(y_true, y_pred, y_proba)
                self._mostrar_metricas(met)

            # >>> CAMBIO: dibujar inmediatamente los datos cargados en el canvas
            self._plot_array(self.plot_buffer)

            QMessageBox.information(self, "Listo", "Archivo procesado, tablas actualizadas y gráfico mostrado.")
        except Exception as e:
            QMessageBox.critical(self, "Error al cargar", str(e))

    def _llenar_tabla_raw(self, df):
        """Muestra vista previa (primeras filas) del DataFrame cargado."""
        max_rows = min(200, df.shape[0])
        cols = df.columns.tolist()
        self.table_raw.setColumnCount(len(cols))
        self.table_raw.setHorizontalHeaderLabels(cols)
        self.table_raw.setRowCount(max_rows)
        for i in range(max_rows):
            for j, c in enumerate(cols):
                val = df.iat[i, j]
                self.table_raw.setItem(i, j, QTableWidgetItem(str(val)))
        self.table_raw.resizeColumnsToContents()

    def action_guardar_datos(self):
        if len(self.raw_data_buffer) == 0:
            QMessageBox.warning(self, "Sin datos", "No hay datos crudos en buffer para guardar.")
            return
        options = QFileDialog.Options()
        fname, _ = QFileDialog.getSaveFileName(self, "Guardar datos crudos como .csv", "", "CSV Files (*.csv);;All Files (*)", options=options)
        if not fname:
            return
        # nombre columnas basado en buffer (ax,ay,az,gx,gy,gz)
        ncols = len(self.raw_data_buffer[0])
        cols = ["ax","ay","az","gx","gy","gz"][:ncols]
        df = pd.DataFrame(self.raw_data_buffer, columns=cols)
        df.to_csv(fname, index=False)
        QMessageBox.information(self, "Guardado", f"Datos guardados en {fname}")

    # -----------------------------------------
    # Widgets: mostrar tabla 
    # -----------------------------------------

    def _llenar_tabla_features(self, X, y_pred=None, y_proba=None):
        """Llena la tabla de features y predicciones (ajusta columnas dinámicamente)."""
        n, m = X.shape
        # columnas: Ventana, Clase pred, Confianza, feat0, feat1, ...
        col_names = ["Ventana", "Clase pred", "Confianza"] + [f"f{i}" for i in range(m)]
        self.table_features.setColumnCount(len(col_names))
        self.table_features.setHorizontalHeaderLabels(col_names)
        self.table_features.setRowCount(n)
        for i in range(n):
            self.table_features.setItem(i, 0, QTableWidgetItem(str(i)))
            if y_pred is not None:
                self.table_features.setItem(i, 1, QTableWidgetItem(str(y_pred[i])))
            else:
                self.table_features.setItem(i, 1, QTableWidgetItem(""))
            if y_proba is not None:
                self.table_features.setItem(i, 2, QTableWidgetItem(f"{np.max(y_proba[i]):.4f}"))
            else:
                self.table_features.setItem(i, 2, QTableWidgetItem(""))

            for j in range(m):
                self.table_features.setItem(i, 3 + j, QTableWidgetItem(f"{X[i,j]:.6f}"))
        self.table_features.resizeColumnsToContents()

    def _mostrar_metricas(self, met):
        txt = ""
        cm = met['confusion_matrix']
        txt += f"Matriz de confusión:\n{cm}\n\n"
        txt += f"Acuracy: {met['accuracy']:.4f}\n"
        txt += f"F1: {met['f1']:.4f}\n"
        txt += f"Precision: {met['precision']:.4f}\n"
        txt += f"Recall: {met['recall']:.4f}\n"
        txt += f"MCC: {met['mcc']:.4f}\n"
        txt += f"AUC: {met['auc']:.4f}\n" if met['auc'] is not None else "AUC: N/A\n"
        self.metrics_text.setPlainText(txt)

    # ----------------------------------------
    # Escaneo sensor
    # ----------------------------------------
    def action_scan_ble(self):
        """Escanea dispositivos BLE (5s) y permite seleccionar uno en un diálogo."""
        if not BLEAK_AVAILABLE:
            QMessageBox.warning(self, "bleak no instalado", "Instala la librería 'bleak' para usar BLE real.")
            return
        import asyncio
        async def discover_once():
            devices = await BleakScanner.discover(timeout=5.0)
            return devices
        try:
            devices = asyncio.run(discover_once())
        except Exception as e:
            QMessageBox.critical(self, "Error escaneo", f"Error al escanear BLE: {e}")
            return

        if not devices:
            QMessageBox.information(self, "Escaneo", "No se detectaron dispositivos BLE.")
            return

        # Construir lista de strings "name - address" (name puede ser None)
        list_items = []
        for d in devices:
            nm = d.name if d.name is not None else "<None>"
            list_items.append(f"{nm} — {d.address}")

        # Mostrar diálogo para seleccionar
        item, ok = QInputDialog.getItem(self, "Dispositivos BLE detectados", "Selecciona dispositivo:", list_items, 0, False)
        if ok and item:
            # extraer la dirección (parte después del guion largo)
            addr = item.split("—")[-1].strip()
            self.selected_device_address = addr
            QMessageBox.information(self, "Seleccionado", f"Dirección seleccionada: {addr}\nSe usará para la próxima conexión.")
        else:
            QMessageBox.information(self, "Cancelado", "No se seleccionó ningún dispositivo.")

    # ----------------------------------------
    # Adquisición y simulación
    # ----------------------------------------

    def action_iniciar_adquisicion(self):
        if self.acquiring:
            QMessageBox.warning(self, "Ya corriendo", "La adquisición ya está en curso.")
            return
        self.acquiring = True
        self.raw_data_buffer = []
        t = threading.Thread(target=self._acquisition_thread, daemon=True)
        t.start()

    def action_detener_adquisicion(self):
        if not self.acquiring:
            QMessageBox.information(self, "No corriendo", "No hay adquisición en curso.")
            return
        self.acquiring = False

    def _acquisition_thread(self):
        """Hilo que gestiona BLE o simulación. Intenta detectar WT901BLE67; lista dispositivos en consola."""
        if self.simulate:
            print("Modo simulación: generando datos sintéticos.")
            t0 = time.time()
            while self.acquiring:
                ts = time.time() - t0
                ax = 0.1 * np.sin(2*np.pi*0.5*ts) + 0.02*np.random.randn()
                ay = 0.1 * np.sin(2*np.pi*0.7*ts + 0.2) + 0.02*np.random.randn()
                az = 9.8 + 0.02*np.random.randn()
                gx = 0.05*np.cos(2*np.pi*0.3*ts) + 0.01*np.random.randn()
                gy = 0.03*np.cos(2*np.pi*0.4*ts) + 0.01*np.random.randn()
                gz = 0.02*np.random.randn()
                sample = [ax, ay, az, gx, gy, gz]
                self.raw_data_buffer.append(sample)
                self.data_queue.put(sample)
                time.sleep(0.02)
            print("Simulación detenida.")
            return

        # BLE real
        if BLEAK_AVAILABLE:
            import asyncio
            async def run_ble():
                try:
                    print("Escaneando dispositivos BLE (5 s)...")
                    devices = await BleakScanner.discover(timeout=5.0)
                    found = False
                    device = None

                    # Si el usuario previamente seleccionó una dirección, intentar usarla primero
                    if self.selected_device_address:
                        addr_user = self.selected_device_address.strip().upper()
                        for d in devices:
                            if d.address and d.address.upper() == addr_user:
                                device = d
                                found = True
                                break
                        if found:
                            print("Usando dispositivo seleccionado por usuario:", device)
                    # Si no hay selección, buscar por coincidencia con BLE_MAC completo o parcial, o BLE_NAME en el nombre
                    if not found:
                        for d in devices:
                            addr = d.address.upper() if d.address else ""
                            name = d.name.lower() if d.name else ""
                            # coincidencia exacta MAC
                            if BLE_MAC and addr == BLE_MAC.upper():
                                device = d
                                found = True
                                break
                            # coincidencia parcial (ultimos 6-8 caracteres)
                            if BLE_MAC and BLE_MAC.replace(":", "")[-6:].upper() in addr.replace(":", ""):
                                device = d
                                found = True
                                break
                            # coincidencia por nombre (subcadena)
                            if BLE_NAME and d.name and BLE_NAME.lower() in name:
                                device = d
                                found = True
                                break

                    # Si todavía no encontrado, imprimir lista y avisar
                    if not found:
                        print("Lista de dispositivos detectados durante escaneo:")
                        for d in devices:
                            print(f" - Name: {d.name}  Address: {d.address}")
                        self.ble_status.emit("No se encontró el sensor automáticamente. Usa 'Escanear BLE' para seleccionar el dispositivo detectado.")
                        return

                    print("Conectando a", device)
                    async with BleakClient(device.address) as client:
                        if not client.is_connected:
                            self.ble_status.emit("No se pudo conectar al sensor (cliente no conectado).")
                            return
                        print("Conectado al sensor:", device)
                        self.ble_status.emit(f"Conectado al sensor (BLE): {device.address}. Intentando subscribir a característica de datos...")

                        # UUIDs candidatos comunes; probar cada uno
                        candidate_uuids = ["0000ffe4-0000-1000-8000-00805f9b34fb",
                                           "0000ffe1-0000-1000-8000-00805f9b34fb",
                                           "0000ffe9-0000-1000-8000-00805f9a34fb"]  # añadí ffe9 por si aplica

                        subscribed = False
                        def callback(sender, data):
                            # intentar parsear datos: 6 floats little-endian o texto con números
                            sample = None
                            try:
                                if len(data) >= 24:
                                    vals = struct.unpack('<6f', data[:24])
                                    sample = [float(v) for v in vals]
                                else:
                                    # intentar decodificar texto
                                    txt = data.decode('utf-8', errors='ignore')
                                    nums = re.findall(r'[-+]?\d*\.\d+|\d+', txt)
                                    if len(nums) >= 6:
                                        sample = [float(x) for x in nums[:6]]
                            except Exception as e:
                                print("Error parseando trama BLE:", e)

                            if sample is not None:
                                self.raw_data_buffer.append(sample)
                                self.data_queue.put(sample)

                        for uuid in candidate_uuids:
                            try:
                                await client.start_notify(uuid, callback)
                                subscribed = True
                                print("Subscribed to", uuid)
                                break
                            except Exception as e:
                                print(f"No se pudo subscribir a {uuid}: {e}")

                        if not subscribed:
                            self.ble_status.emit("No se pudo subscribir a ninguna característica (UUIDs probados). Revisa documentación del sensor o usa 'Escanear BLE' para verificar características.")
                            return

                        # Mantener mientras acquiring sea True
                        while self.acquiring:
                            await asyncio.sleep(0.1)

                        # detener notify (intentar para cada uuid)
                        for uuid in candidate_uuids:
                            try:
                                await client.stop_notify(uuid)
                            except Exception:
                                pass

                except Exception as e:
                    print("Error BLE:", e)
                    self.ble_status.emit(f"Error BLE: {e}")

            try:
                asyncio.run(run_ble())
            except Exception as e:
                print("Error al ejecutar BLE:", e)
                self.ble_status.emit(f"Error BLE (run): {e}")
        else:
            print("bleak no disponible; no se puede conectar por BLE.")
            self.ble_status.emit("bleak no está instalado. El modo BLE usará simulación. Instala 'bleak' si quieres conectar hardware.")

    def actualizar_grafico_desde_cola(self):
        """Toma datos de la cola y actualiza el gráfico (ax y gx)."""
        # primero extraer datos de la cola (adquisición en vivo)
        new_data = []
        while not self.data_queue.empty():
            try:
                s = self.data_queue.get_nowait()
                new_data.append(s)
            except Exception:
                break
        if new_data:
            arr = np.array(new_data)
            if self.plot_buffer is None:
                self.plot_buffer = arr
            else:
                try:
                    self.plot_buffer = np.vstack([self.plot_buffer, arr])
                except Exception:
                    self.plot_buffer = arr
            # limitar tamaño del buffer para no sobrecargar plotting
            if self.plot_buffer.shape[0] > 2000:
                self.plot_buffer = self.plot_buffer[-2000:, :]

        # Si no hay nada para plotear, salir
        if self.plot_buffer is None:
            return

        # Dibujar usando rutina reutilizable
        self._plot_array(self.plot_buffer)

    # >>> CAMBIO: función genérica para plotear arrays (N, n_channels)
    def _plot_array(self, arr):
        """
        Dibuja los canales más relevantes:
         - si hay >=3 canales -> plot ax, ay, az (canales 0..2)
         - si hay >=6 canales -> además plot gx (3), gy (4), gz (5)
         - si hay menos canales, dibuja las primeras 2 columnas
        """
        self.canvas.axes.cla()
        n_ch = arr.shape[1]
        x = np.arange(arr.shape[0])
        plotted = False
        # acelerómetros
        if n_ch >= 3:
            self.canvas.axes.plot(x, arr[:,0], label='ax')
            self.canvas.axes.plot(x, arr[:,1], label='ay')
            self.canvas.axes.plot(x, arr[:,2], label='az')
            plotted = True
        else:
            # dibujar hasta dos primeras columnas si existen
            if n_ch >= 1:
                self.canvas.axes.plot(x, arr[:,0], label='ch0')
                plotted = True
            if n_ch >= 2:
                self.canvas.axes.plot(x, arr[:,1], label='ch1')
                plotted = True
        # giroscopios
        if n_ch >= 6:
            # dibujar en el mismo eje (puedes cambiar a subplots si prefieres)
            self.canvas.axes.plot(x, arr[:,3], label='gx')
            self.canvas.axes.plot(x, arr[:,4], label='gy')
            self.canvas.axes.plot(x, arr[:,5], label='gz')
            plotted = True

        if not plotted:
            self.canvas.axes.text(0.5, 0.5, "No hay canales para graficar", transform=self.canvas.axes.transAxes, ha='center')

        self.canvas.axes.legend(loc='upper right', fontsize='small')
        self.canvas.axes.set_title("Señales (datos cargados / en vivo)")
        self.canvas.draw()

    # ------------------------------------------
    # Procesar bloque adquirido 
    # ------------------------------------------

    def action_procesar_bloque(self):
        if len(self.raw_data_buffer) == 0:
            QMessageBox.warning(self, "Sin bloque", "No hay bloque adquirido.")
            return
        data = np.array(self.raw_data_buffer)
        window_size = int(self.input_window.text())
        step = int(self.input_step.text())
        X = extract_features(data, window_size=window_size, step=step)
        model = self.model
        if model is None:
            QMessageBox.warning(self, "Sin modelo", "Primero entrena o carga un modelo (botón 'Entrenar modelo').")
            return
        if X.shape[0] == 0:
            QMessageBox.warning(self, "Sin ventanas", "No se extrajeron ventanas (revisa window/step).")
            return
        y_proba = model.predict_proba(X)
        y_pred = model.predict(X)
        self._llenar_tabla_features(X, y_pred, y_proba)
        clases_unicas, counts = np.unique(y_pred, return_counts=True)
        conf_prom = np.mean(np.max(y_proba, axis=1))
        txt = f"Predicciones por clase: {dict(zip(clases_unicas, counts))}\nConfianza promedio: {conf_prom:.3f}"
        self.metrics_text.setPlainText(txt)
        QMessageBox.information(self, "Procesado", "Bloque procesado con el modelo.")

    # ------------------------------------------
    # Entrenar 
    # ------------------------------------------

    def action_entrenar_modelo(self):
        """Genera datos sintéticos, entrena un SVM y lo guarda como ejemplo."""
        N = 2000
        t = np.linspace(0, 20, N)
        ax0 = 0.02*np.random.randn(N)
        ax1 = 0.3*np.sin(2*np.pi*1.0*t) + 0.05*np.random.randn(N)
        data0 = np.column_stack([ax0, 0.02*np.random.randn(N), 9.8+0.02*np.random.randn(N),
                                 0.01*np.random.randn(N), 0.01*np.random.randn(N), 0.01*np.random.randn(N)])
        data1 = np.column_stack([ax1, 0.02*np.random.randn(N), 9.8+0.02*np.random.randn(N),
                                 0.01*np.random.randn(N), 0.01*np.random.randn(N), 0.01*np.random.randn(N)])
        concat = np.vstack([data0[:1000], data1[:1000]])
        labels = np.hstack([np.zeros(1000), np.ones(1000)])
        window_size = int(self.input_window.text())
        step = int(self.input_step.text())
        X = extract_features(concat, window_size=window_size, step=step)
        wins = sliding_window(labels.reshape(-1,1), window_size, step)
        y_win = np.array([stats.mode(w.flatten())[0][0] for w in wins])
        kbest = int(self.input_kbest.text())
        pipe = crear_y_entrenar_modelo(X, y_win, k_best=kbest)
        self.model = pipe
        QMessageBox.information(self, "Entrenado", f"Modelo entrenado y guardado en {MODEL_PATH}")

# -----------------------------------------------------------------------------
# Ejecucion
# -----------------------------------------------------------------------------

def main():
    app = QtWidgets.QApplication(sys.argv)
    win = MainWindow()
    sys.exit(app.exec_())

if __name__ == "__main__":
    main()


No se encontró modelo entrenado. Use 'Entrenar modelo' o cargue uno.


C:\Users\aryga\AppData\Local\Temp\ipykernel_31208\3889033506.py:497: RuntimeWarning: coroutine 'MainWindow.action_scan_ble.<locals>.discover_once' was never awaited
  return


Escaneando dispositivos BLE (5 s)...
Lista de dispositivos detectados durante escaneo:
 - Name: None  Address: 31:4E:9F:51:8D:7E
